# Modular Financial Agent Walkthrough

## Advanced Multi-Tool Coordination Using Helper Modules Architecture

This notebook provides a comprehensive walkthrough of the **Modular Financial Agent** system, demonstrating how the helper modules work together to create a sophisticated financial analysis platform. The system features:

- **🏗️ Modular Architecture**: Clean separation using helper modules
- **📄 Document Analysis**: SEC 10-K filing analysis via DocumentToolsManager
- **🔧 Function Tools**: Database queries, market data, and PII protection via FunctionToolsManager  
- **🧠 Intelligent Coordination**: AgentCoordinator with smart routing and automatic PII protection
- **🔄 Seamless Integration**: All components work together automatically

### 🏛️ Architecture Overview

The system uses a **3-module architecture**:

#### **1. DocumentToolsManager** (`helper_modules/document_tools.py`)
- Creates 3 QueryEngineTool objects for AAPL, GOOGL, TSLA 10-K filings
- Each tool analyzes specific company SEC documents
- Powered by LlamaIndex with OpenAI embeddings

#### **2. FunctionToolsManager** (`helper_modules/function_tools.py`)  
- Creates 3 function tools for different capabilities:
  - `database_query_tool` - SQL generation and execution with column output
  - `finance_market_search_tool` - Real-time Yahoo Finance data
  - `pii_protection_tool` - Automatic sensitive data masking

#### **3. AgentCoordinator** (`helper_modules/agent_coordinator.py`)
- Orchestrates all tools with intelligent routing
- Automatically detects when PII protection is needed
- Synthesizes results from multiple tools
- Provides clean backward compatibility

Let's explore how this modular system delivers comprehensive financial insights!

## 1. Environment Setup and Module Imports

First, let's set up our environment and import the modular components. The beauty of this architecture is that each module can be used independently or together.

In [ ]:
# Import required libraries
import os
import sys
from pathlib import Path
import pandas as pd

# Add current directory to Python path for imports
current_dir = Path().absolute()
if str(current_dir) not in sys.path:
    sys.path.append(str(current_dir))

# Import the modular helper modules
from helper_modules.document_tools import DocumentToolsManager
from helper_modules.function_tools import FunctionToolsManager  
from helper_modules.agent_coordinator import AgentCoordinator

print("✅ All helper modules imported successfully!")
print(f"📁 Working directory: {current_dir}")

# Verify required files exist
required_files = [
    "data/financial.db",
    "data/10k_documents/AAPL_10K_2024.pdf",
    "data/10k_documents/GOOGL_10K_2024.pdf", 
    "data/10k_documents/TSLA_10K_2024.pdf"
]

print("\n🔍 Checking required files:")
all_files_exist = True
for file_path in required_files:
    exists = Path(file_path).exists()
    status = "✅" if exists else "❌"
    print(f"   {status} {file_path}")
    if not exists:
        all_files_exist = False

print(f"\n💡 OpenAI API Key configured: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")
print(f"📋 System ready: {'✅' if all_files_exist else '❌'}")

## 2. Module 1: DocumentToolsManager - PDF Document Analysis

Let's start by exploring the `DocumentToolsManager`. This module creates specialized tools for analyzing SEC 10-K filings for each company.

In [ ]:
# Initialize the DocumentToolsManager
print("📄 Initializing DocumentToolsManager...")
print("=" * 50)

doc_manager = DocumentToolsManager()
print(f"✅ DocumentToolsManager created")

# Build the document tools
print("\n🔧 Building document analysis tools...")
document_tools = doc_manager.build_document_tools()

print(f"\n📊 Document Tools Created:")
for i, tool in enumerate(document_tools, 1):
    print(f"   {i}. {tool.metadata.name}")
    print(f"      Description: {tool.metadata.description}")
    print(f"      Type: {type(tool).__name__}")
    print()

print(f"🎯 Total Document Tools: {len(document_tools)}")
print("=" * 50)

In [ ]:
# Test individual document tools
print("🧪 Testing Document Tools Individually")
print("=" * 50)

# Test Apple 10-K analysis
print("🍎 Testing Apple 10-K Tool:")
apple_tool = document_tools[0]  # First tool should be Apple
apple_query = "What are Apple's main revenue sources according to their 10-K filing?"

print(f"Query: {apple_query}")
try:
    apple_result = apple_tool.query_engine.query(apple_query)
    print(f"Result: {str(apple_result)[:300]}...")
    print("✅ Apple document tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "-" * 40)

# Test Google 10-K analysis  
print("🌐 Testing Google 10-K Tool:")
google_tool = document_tools[1]  # Second tool should be Google
google_query = "What business segments does Google operate in?"

print(f"Query: {google_query}")
try:
    google_result = google_tool.query_engine.query(google_query)
    print(f"Result: {str(google_result)[:300]}...")
    print("✅ Google document tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("=" * 40)

# Test Tesla 10-K analysis
print("🚗 Testing Tesla 10-K Tool:")
tesla_tool = document_tools[2]  # Third tool should be Tesla
tesla_query = "What are Tesla's main revenue sources according to their 10-K filing?"

print(f"Query: {tesla_query}")
try:
    tesla_result = tesla_tool.query_engine.query(tesla_query)
    print(f"Result: {str(tesla_result)[:300]}...")
    print("✅ Tesla document tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("=" * 50)

## 3. Module 2: FunctionToolsManager - Dynamic Function Tools

Now let's explore the `FunctionToolsManager`. This module creates three powerful function tools for database queries, market data, and PII protection.

In [ ]:
# Initialize the FunctionToolsManager
print("🔧 Initializing FunctionToolsManager...")
print("=" * 50)

func_manager = FunctionToolsManager()
print(f"✅ FunctionToolsManager created")
print(f"💾 Database path: {func_manager.db_path}")

# Create the function tools
print("\n🛠️ Creating function tools...")
function_tools = func_manager.create_function_tools()

print(f"\n📊 Function Tools Created:")
for i, tool in enumerate(function_tools, 1):
    print(f"   {i}. {tool.metadata.name}")
    print(f"      Description: {tool.metadata.description}")
    print(f"      Function: {tool.fn.__name__}")
    print()

print(f"🎯 Total Function Tools: {len(function_tools)}")
print("=" * 50)

In [ ]:
# Test individual function tools
print("🧪 Testing Function Tools Individually")
print("=" * 50)

# Test Database Query Tool
print("💾 Testing Database Query Tool:")
db_tool = next(tool for tool in function_tools if "database" in tool.metadata.name)
db_query = "How many customers are in the database?"

print(f"Query: {db_query}")
try:
    db_result = db_tool.fn(db_query)
    print(f"Result: {db_result}")
    print("✅ Database tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "-" * 40)

# Test Market Data Tool
print("📈 Testing Market Data Tool:")
market_tool = next(tool for tool in function_tools if "market" in tool.metadata.name)
market_query = "What is Apple's current stock price?"

print(f"Query: {market_query}")
try:
    market_result = market_tool.fn(market_query)
    print(f"Result: {market_result}")
    print("✅ Market data tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("\n" + "-" * 40)

# Test PII Protection Tool
print("🔒 Testing PII Protection Tool:")
pii_tool = next(tool for tool in function_tools if "pii" in tool.metadata.name)
test_data = "Customer: John Doe, Email: john.doe@example.com, Phone: 555-1234"
test_columns = "['customer_name', 'email', 'phone']"

print(f"Test Data: {test_data}")
print(f"Columns: {test_columns}")
try:
    pii_result = pii_tool.fn(test_data, test_columns)
    print(f"Protected Result: {pii_result}")
    print("✅ PII protection tool working!")
except Exception as e:
    print(f"❌ Error: {e}")

print("=" * 50)

## 4. Module 3: AgentCoordinator - Intelligent Orchestration

Now let's see the real power - the `AgentCoordinator` that intelligently orchestrates all tools together, providing automatic routing and PII protection coordination.

In [ ]:
# Initialize the AgentCoordinator
print("🧠 Initializing AgentCoordinator...")
print("=" * 50)

# Create coordinator with verbose output to see the process
coordinator = AgentCoordinator(verbose=True)

# Setup the coordinator - it will automatically create all tools
print("\n🔧 Setting up coordinator with automatic tool creation...")
coordinator.setup()

# Get comprehensive status
status = coordinator.get_status()
print(f"\n📊 Coordinator Status:")
for key, value in status.items():
    print(f"   {key}: {value}")

print("=" * 50)

In [ ]:
# Explore the created tools within the coordinator
print("🔍 Exploring Coordinator's Tool Ecosystem")
print("=" * 50)

# Show document tools
print("📄 Document Tools in Coordinator:")
for i, tool in enumerate(coordinator.document_tools, 1):
    print(f"   {i}. {tool.metadata.name}")
    print(f"      {tool.metadata.description}")

print(f"\n🔧 Function Tools in Coordinator:")
for i, tool in enumerate(coordinator.function_tools, 1):
    print(f"   {i}. {tool.metadata.name}")
    print(f"      {tool.metadata.description}")

# Show available tools summary
available = coordinator.get_available_tools()
print(f"\n📋 Available Tools Summary:")
print(f"   Document Tools: {available['document_tools']}")
print(f"   Function Tools: {available['function_tools']}")
print(f"   Total Tools: {available['total_tools']}")

print("=" * 50)

## 5. Single Tool Routing - Document Analysis

Let's test how the coordinator intelligently routes queries to individual tools. We'll start with document-based queries.

In [ ]:
# Test document tool routing
print("📄 Testing Document Tool Routing")
print("=" * 50)

# Apple-specific query
print("🍎 Apple 10-K Analysis:")
apple_query = "What are Apple's main business segments and how do they generate revenue?"
print(f"Query: {apple_query}")

apple_response = coordinator.query(apple_query, verbose=True)
print(f"\nFinal Response: {apple_response}")
print("\n" + "=" * 50)

# Google-specific query
print("🌐 Google 10-K Analysis:")
google_query = "What are the key risk factors mentioned in Google's 10-K filing?"
print(f"Query: {google_query}")

google_response = coordinator.query(google_query, verbose=True)
print(f"\nFinal Response: {google_response}")
print("\n" + "=" * 50)

## 6. Single Tool Routing - Function Tools

Now let's test how the coordinator routes to function tools for database queries and market data.

In [ ]:
# Test database tool routing
print("💾 Testing Database Tool Routing")
print("=" * 50)

# Database query without PII
print("📊 Non-PII Database Query:")
db_query = "How many customers are in the database grouped by investment profile?"
print(f"Query: {db_query}")

db_response = coordinator.query(db_query, verbose=True)
print(f"\nFinal Response: {db_response}")
print("\n" + "=" * 50)

# Market data query
print("📈 Testing Market Data Tool Routing:")
market_query = "What is Tesla's current stock price and trading volume?"
print(f"Query: {market_query}")

market_response = coordinator.query(market_query, verbose=True)
print(f"\nFinal Response: {market_response}")
print("=" * 50)

## 7. Automatic PII Protection Coordination

One of the most sophisticated features is the automatic PII protection. Let's test how the coordinator automatically detects when database results contain PII and applies protection.

In [ ]:
# Test automatic PII protection coordination
print("🔒 Testing Automatic PII Protection Coordination")
print("=" * 60)

# Query that will return PII data
print("🧪 Test: Query WITH PII Fields (should trigger automatic protection)")
print("-" * 50)

pii_query = "Show me customer details including names and emails for customers who own Apple stock"
print(f"Query: {pii_query}")

pii_response = coordinator.query(pii_query, verbose=True)
print(f"\nFinal Response: {pii_response}")

print("\n" + "=" * 60)

# Query that doesn't return PII data
print("🧪 Test: Query WITHOUT PII Fields (should NOT trigger protection)")
print("-" * 50)

no_pii_query = "Show portfolio values and investment profiles without customer details"
print(f"Query: {no_pii_query}")

no_pii_response = coordinator.query(no_pii_query, verbose=True)
print(f"\nFinal Response: {no_pii_response}")

print("\n🎯 Key Observations:")
print("✅ PII protection automatically applied when needed")
print("✅ No protection overhead for non-PII queries")
print("✅ Clean separation between data retrieval and privacy protection")
print("=" * 60)

## 8. Multi-Tool Coordination - The Real Power

Now let's see the coordinator's most impressive capability: intelligently combining multiple tools to answer complex queries that require different data sources.

In [ ]:
# Test dual tool coordination: Database + Market Data
print("🔄 Testing Dual Tool Coordination: Database + Market Data")
print("=" * 60)

dual_query = "Compare the current Tesla stock price with what our Tesla-holding customers paid based on portfolio data"
print(f"Query: {dual_query}")

dual_response = coordinator.query(dual_query, verbose=True)
print(f"\nFinal Response: {dual_response}")
print("=" * 60)

In [ ]:
# Test triple tool coordination: Document + Database + Market
print("🌟 Testing Triple Tool Coordination: Document + Database + Market")
print("=" * 60)

triple_query = "Analyze Apple comprehensively: show me Apple's business segments from their 10-K, current stock performance, and which of our customers own Apple stock"
print(f"Query: {triple_query}")

triple_response = coordinator.query(triple_query, verbose=True)
print(f"\nFinal Response: {triple_response}")
print("=" * 60)

## 9. Advanced Database Queries with Automatic SQL Generation

Let's explore the sophisticated database capabilities including automatic SQL generation and smart result handling.

In [ ]:
# Test complex database queries
print("🎯 Testing Advanced Database Capabilities")
print("=" * 50)

# Complex aggregation query
print("📊 Complex Aggregation Query:")
complex_query = "What's the average portfolio value for each investment profile and how many customers are in each category?"
print(f"Query: {complex_query}")

complex_response = coordinator.query(complex_query, verbose=True)
print(f"\nResponse: {complex_response}")

print("\n" + "-" * 40)

# Multi-table join query
print("🔗 Multi-table Join Query:")
join_query = "Show the top 5 customers by total portfolio value along with their stock holdings details"
print(f"Query: {join_query}")

join_response = coordinator.query(join_query, verbose=True)
print(f"\nResponse: {join_response}")

print("=" * 50)

## 10. System Architecture Analysis

Let's analyze what makes this modular architecture so powerful and examine the internal coordination mechanisms.

In [ ]:
# Analyze the system architecture
print("🏗️ System Architecture Analysis")
print("=" * 50)

print("📊 Module Breakdown:")
print(f"   DocumentToolsManager: {len(coordinator.document_tools)} tools")
print(f"   FunctionToolsManager: {len(coordinator.function_tools)} tools")
print(f"   AgentCoordinator: 1 orchestrator")

print(f"\n🧠 Coordinator Intelligence Features:")
print("   ✅ Automatic tool selection via LLM routing")
print("   ✅ Multi-tool result synthesis")
print("   ✅ Automatic PII detection and protection")
print("   ✅ Error handling and graceful fallbacks")
print("   ✅ Clean modular architecture")

print(f"\n🔄 Tool Coordination Flow:")
print("   1. Query received by AgentCoordinator")
print("   2. LLM analyzes query and selects appropriate tools")
print("   3. Selected tools execute in parallel/sequence")
print("   4. Database results checked for PII fields")
print("   5. PII protection applied automatically if needed")
print("   6. Results synthesized into final response")

print(f"\n🎯 Key Architecture Benefits:")
print("   ✅ Clean separation of concerns")
print("   ✅ Each module can be used independently")
print("   ✅ Easy to test, maintain, and extend")
print("   ✅ Automatic coordination reduces complexity")
print("   ✅ Backward compatibility with existing code")

print("=" * 50)

## Summary and Key Takeaways

### 🎉 Modular Architecture Achievements

This walkthrough demonstrated the power of the **helper modules architecture**:

#### 🏗️ **Clean Modular Design**
- **DocumentToolsManager**: Focused solely on PDF document analysis
- **FunctionToolsManager**: Handles database, market data, and PII protection
- **AgentCoordinator**: Intelligent orchestration and routing

#### 🧠 **Intelligent Coordination**
- **Automatic Tool Selection**: LLM-based routing to appropriate tools
- **Multi-Tool Synthesis**: Seamless combination of results from different sources
- **Automatic PII Protection**: Smart detection and protection of sensitive data
- **Error Handling**: Graceful degradation and robust error management

#### 🔄 **Flexibility and Reusability**
- **Independent Modules**: Each can be used standalone or together
- **Easy Testing**: Clean interfaces enable comprehensive unit testing
- **Maintainable Code**: Separation of concerns makes updates simple
- **Extensible Design**: Easy to add new tools or capabilities

### 🎯 **Query Types Successfully Demonstrated**

1. **Single Tool Queries**
   - Document analysis (10-K filings)
   - Database queries with automatic SQL generation
   - Real-time market data retrieval

2. **Dual Tool Coordination**
   - Database + Market data combination
   - Document + Database synthesis

3. **Tool Orchestration**
   - Document + Database + Market comprehensive analysis

4. **Automatic PII Protection**
   - Field-based detection and masking
   - Zero performance impact for non-PII queries
   - Transparent coordination between tools